In [ ]:
import nest_asyncio
nest_asyncio.apply()
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sys
sys.path.insert(0, '..')
from bundle import evaluate
from bundle.fpga_config import FPGA_config
from bundle.fpga_dashboard import Dashboard
from bundle.fpga_simu import Simu
from bundle.fpga_device import Device
from pyclk import Trace

In [ ]:
mem_nb = 16
iter_nb = mem_nb // 2
ddr2fpga_nb = 4
fpga2ddr_nb = 2
mem_depth = 1024
add_nb = 4
mul_nb = 4

config = FPGA_config(ddr2fpga_nb, fpga2ddr_nb, iter_nb, mem_nb, mem_depth, add_nb, mul_nb)
fpga = Simu(config)

debug = True
db = Dashboard(config.config)
device = Device(config, fpga, debug=debug, dashboard=db)

In [ ]:
trace = Trace()
for i in range(config.config['iter_nb']):
    trace.add(fpga.u_iter[i].o_raddr)
    trace.add(fpga.u_iter[i].o_waddr)
    trace.add(fpga.u_iter[i].o_wena)
    trace.add(fpga.s_iter_rmem0_i[i])
    trace.add(fpga.s_iter_rmem1_i[i])
    trace.add(fpga.s_iter_wmem_i[i])
for i in range(config.config['mem_nb']):
    trace.add(fpga.s_mem_addr[i])
    trace.add(fpga.s_mem_wena[i])
    trace.add(fpga.s_mem_din[i])
    trace.add(fpga.s_mem_dout[i])
fpga.set_trace(trace)

In [ ]:
size = 10000
a0 = np.arange(size).astype(np.uint64)
a1 = np.ones_like(a0)
a2 = a1 * 2
a3 = a1 * 3

expr = 'a0 * a1 + a2 * a3'

In [ ]:
res = evaluate(expr, device, debug=debug)

In [ ]:
res

In [ ]:
db.show()

In [ ]:
# FPGA runs at 100 MHz
print('In FPGA:')
print(f'{fpga.time * 1e6 / 1e8} us')

In [ ]:
cexpr = compile(expr, 'fakemodule', 'exec')
print('In CPU:')
%timeit exec(cexpr)

In [ ]:
#res_true = evaluate(expr)
res_true = eval(expr)
res_true

In [ ]:
if np.array_equal(res, res_true):
    print('Success!')
else:
    print('Failed!')

In [ ]:
res = np.array(res)
diff_idx = np.where(res != res_true)
plt.figure(figsize=(15, 5))
#plt.scatter(np.arange(res_true.size), res_true, c='b')
plt.scatter(diff_idx, res[diff_idx], c='r', label='FPGA')
plt.scatter(diff_idx, res_true[diff_idx], c='g', label='NumPy')
#plt.scatter(diff_idx, z_true[diff_idx]-z[diff_idx], c='r')
#plt.xlim(0, size)
#plt.ylim(1, size+1)
plt.legend(loc='upper left')
plt.title('Differences')
plt.show()

In [ ]:
res[diff_idx]

In [ ]:
res_true[diff_idx]

In [ ]:
trace.plot()